In [1]:
%use kandy, multik
import generator.Generator
import generator.Point

In [2]:
import java.text.DecimalFormat

val repeats = 10000
var blocks = 0
repeat(repeats) {
    val (p1, p2) = Generator.nextPariInSquare(h, A)
    val obstacles = HashSet<Point>()
    repeat(poissonDistribution.sample()) {
        obstacles.add(Generator.nextPointInSquare(A))
    }
    var block = false
    check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
        for (p in obstacles) {
            if (i.isInCircle(p, r)) {
                block = true
                break@check
            }
        }
    }
    if (block) {
        ++blocks
    }
}
println("Probability: ${DecimalFormat("#0.00#").format(blocks / repeats.toDouble())}")

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[2], line 6, column 47: Unresolved reference: h
at Cell In[2], line 6, column 50: Unresolved reference: A
at Cell In[2], line 8, column 12: Unresolved reference: poissonDistribution
at Cell In[2], line 9, column 51: Unresolved reference: A
at Cell In[2], line 14, column 33: Unresolved reference. None of the following candidates is applicable because of receiver type mismatch: 
public val Int.r: RInt defined in org.jetbrains.kotlinx.multik.ndarray.data

In [15]:
fun mathModel(a: Number, h: Number, r: Number): Double {
    return min(2 * r.toDouble() * h.toDouble() * a.toDouble(), 1.0)
}

In [18]:
import org.apache.commons.math3.distribution.PoissonDistribution
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

val hs = listOf<Number>(1, 4, 8) // point distance
val `as` = listOf<Number>(0.01, 0.05, 0.1, 0.3) // obstacle density
val rs = listOf<Number>(0.05, 0.1, 0.5) // obstacle size
val df = DecimalFormat("#0.00#")

println("a\th\tr\tImitation\tModel\tCV")
var cvsum = 0.0
var cvcount = 0
for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            val imitation = blocks / repeats.toDouble()
            val mathModel = mathModel(a, h, r)
            val diff = imitation - mathModel
            val cv = diff.pow(2) / (abs(if (diff == 0.0) 1.0 else diff) / 2)
            cvsum += cv
            ++cvcount
            println("$a\t$h\t$r\t${df.format(imitation)}\t${df.format(mathModel)}\t${df.format(cv)}")
        }
    }
}
println("cv mean = ${df.format(cvsum / cvcount)}")

a	h	r	Imitation	Model	CV
0.01	1	0.05	0.001	0.001	0.00
0.01	1	0.1	0.00	0.002	0.004
0.01	1	0.5	0.02	0.01	0.02
0.01	4	0.05	0.004	0.004	0.00
0.01	4	0.1	0.009	0.008	0.002
0.01	4	0.5	0.04	0.04	0.00
0.01	8	0.05	0.01	0.008	0.004
0.01	8	0.1	0.02	0.016	0.008
0.01	8	0.5	0.085	0.08	0.01
0.05	1	0.05	0.003	0.005	0.004
0.05	1	0.1	0.016	0.01	0.012
0.05	1	0.5	0.087	0.05	0.074
0.05	4	0.05	0.02	0.02	0.00
0.05	4	0.1	0.045	0.04	0.01
0.05	4	0.5	0.201	0.20	0.002
0.05	8	0.05	0.044	0.04	0.008
0.05	8	0.1	0.082	0.08	0.004
0.05	8	0.5	0.338	0.40	0.124
0.1	1	0.05	0.012	0.01	0.004
0.1	1	0.1	0.025	0.02	0.01
0.1	1	0.5	0.144	0.10	0.088
0.1	4	0.05	0.036	0.04	0.008
0.1	4	0.1	0.078	0.08	0.004
0.1	4	0.5	0.399	0.40	0.002
0.1	8	0.05	0.086	0.08	0.012
0.1	8	0.1	0.147	0.16	0.026
0.1	8	0.5	0.584	0.80	0.432
0.3	1	0.05	0.024	0.03	0.012
0.3	1	0.1	0.07	0.06	0.02
0.3	1	0.5	0.428	0.30	0.256
0.3	4	0.05	0.098	0.12	0.044
0.3	4	0.1	0.226	0.24	0.028
0.3	4	0.5	0.725	1.00	0.55
0.3	8	0.05	0.207	0.24	0.066
0.3	8	0.1	0.361	0.48	0.238
0.3	8	0.5	

In [21]:
import org.apache.commons.math3.distribution.PoissonDistribution

val A = 10 // square side

val h = 3 // point distance
val a = 0.2 // obstacle density
val r = 0.1 // obstacle size

val poissonDistribution = PoissonDistribution(a * A * A)

val (p1, p2) = Generator.nextPariInSquare(h, A)
val obstacles = HashSet<Point>()
repeat(poissonDistribution.sample()) {
    obstacles.add(Generator.nextPointInSquare(A))
}

val psize = 25
plot {
    
    points { 
        x(obstacles.map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(obstacles.map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        size = psize.toDouble() * r
    }

    points {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        color = Color.RED
    }

    line {
        x(listOf(p1, p2).map { it.x }) {
            scale = continuous(0.0, A.toDouble())
        }
        y(listOf(p1, p2).map { it.y }) {
            scale = continuous(0.0, A.toDouble())
        }
        type = LineType.DASHED
        color = Color.RED
    }

    layout {
        size = Pair(A * psize, A * psize)
        style { 
            axis {
                blankAxes()
            }
        }
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="CKCTcJ"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":250.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
},{
"aesthetic":"x",
"limits":[0.0,10.0]
},{
"aesthetic":"y",
"limits":[0.0,10.0]
}],
"layers":[{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[3.6990532870911186,1.6915124120617653,3.126488255298846,3.319247322669394,3.592945853541394,9.279826572697218,4.876758774381269,5.4558695648082045,0.6358842474027737,0.0971251793326755,1.351773924307107,8.473141079408883,6.249701190844076,8.938015268217491,8.625422278517547,4.30190327476722,0.9163793407756393,0.6669209044256141],
"y":[0.7331154362412817,7.091433322500269,9.522872867843382,2.964172197049151,5.676749899511862,7.575783425300164,0.04697139585896459,3.789711498714028,2.8955047575959325,3.5859549468732266,8.134308252179224,6.302473896622739,4.103378706307095,3.647478557152186,2.295100346524012,3.4903007737556244,2.2224082903015487,3.480188907394366]
},
"size":2.5,
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[3.75664826889943,6.243351731100571],
"y":[5.839092648501472,4.160907351498528]
},
"color":"#ee6666",
"sampling":"none",
"position":"identity",
"geom":"point"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":[3.75664826889943,6.243351731100571],
"y":[5.839092648501472,4.160907351498528]
},
"color":"#ee6666",
"linetype":"dashed",
"sampling":"none",
"position":"identity",
"geom":"line"
}],
"theme":{
"axis":"blank",
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("CKCTcJ");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>">

In [26]:
import org.apache.commons.math3.distribution.PoissonDistribution
import org.jetbrains.kotlinx.dataframe.api.dataFrameOf
import java.text.DecimalFormat

val repeats = 1000

val A = 10 // square side

//val hs = (1..100 step 1).map { it / 10.0 } // point distance
//val `as` = listOf(0.1) // obstacle density
//val rs = listOf(0.1) // obstacle size

val hs = listOf(3) // point distance
val `as` = listOf(0.1) // obstacle density
val rs = (1..1000 step 10).map { it / 1000.0 } // obstacle size

val ps = ArrayList<Double>()

for (a in `as`) {
    val poissonDistribution = PoissonDistribution(a.toDouble() * A * A)
    for (h in hs) {
        for (r in rs) {
            var blocks = 0
            repeat(repeats) {
                val (p1, p2) = Generator.nextPariInSquare(h, A)
                val obstacles = HashSet<Point>()
                repeat(poissonDistribution.sample()) {
                    obstacles.add(Generator.nextPointInSquare(A))
                }
                var block = false
                check@for (i in p1.interpolate(p2, round(p1.distTo(p2) / 0.01).toInt())) {
                    for (p in obstacles) {
                        if (i.isInCircle(p, r)) {
                            block = true
                            break@check
                        }
                    }
                }
                if (block) {
                    ++blocks
                }
            }
            ps.add(blocks / repeats.toDouble())
        }
    }
}

dataFrameOf(
    "r" to `rs`.toList(),
    "p" to ps
).plot { 
    line {
        x("r")
        y("p")
    }
    layout {
        size = Pair(2 * A * psize, A * psize)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ZNnrIN"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"p":[0.0,0.006,0.005,0.019,0.028,0.034,0.043,0.034,0.049,0.059,0.067,0.065,0.072,0.062,0.089,0.091,0.094,0.122,0.108,0.12,0.117,0.13,0.156,0.132,0.143,0.156,0.158,0.174,0.181,0.159,0.185,0.188,0.192,0.217,0.205,0.214,0.229,0.245,0.261,0.222,0.237,0.266,0.284,0.244,0.291,0.289,0.299,0.304,0.296,0.298,0.326,0.305,0.313,0.351,0.315,0.346,0.32,0.353,0.357,0.336,0.357,0.376,0.402,0.382,0.405,0.401,0.399,0.429,0.453,0.426,0.42,0.422,0.427,0.42,0.45,0.452,0.476,0.497,0.466,0.471,0.468,0.475,0.501,0.511,0.524,0.487,0.52,0.537,0.506,0.534,0.549,0.548,0.53,0.539,0.545,0.558,0.547,0.559,0.602,0.585],
"r":[0.001,0.011,0.021,0.031,0.041,0.051,0.061,0.071,0.081,0.091,0.101,0.111,0.121,0.131,0.141,0.151,0.161,0.171,0.181,0.191,0.201,0.211,0.221,0.231,0.241,0.251,0.261,0.271,0.281,0.291,0.301,0.311,0.321,0.331,0.341,0.351,0.361,0.371,0.381,0.391,0.401,0.411,0.421,0.431,0.441,0.451,0.461,0.471,0.481,0.491,0.501,0.511,0.521,0.531,0.541,0.551,0.561,0.571,0.581,0.591,0.601,0.611,0.621,0.631,0.641,0.651,0.661,0.671,0.681,0.691,0.701,0.711,0.721,0.731,0.741,0.751,0.761,0.771,0.781,0.791,0.801,0.811,0.821,0.831,0.841,0.851,0.861,0.871,0.881,0.891,0.901,0.911,0.921,0.931,0.941,0.951,0.961,0.971,0.981,0.991]
},
"ggsize":{
"width":500.0,
"height":250.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null]
}],
"layers":[{
"mapping":{
"x":"r",
"y":"p"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"line",
"data":{
}
}]
};
 var plotContainer = document.getElementById("ZNnrIN");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 <path d="M19.862059035441753 182.3181818181818 L19.862059035441753 182.3181818181818 L23.874596214318874 180.58758683177288 L27.887133393195995 180.8760193295077 L31.89967057207312 176.83796436122017 L35.912207750950245 174.24207188160676 L39.92474492982736 172.51147689519783 L43.93728210870448 169.91558441558442 L47.949819287581604 172.51147689519783 L51.96235646645873 168.18498942917546 L55.97489364533585 165.30066445182723 L59.98743082421297 162.99320446994864 L63.999968003090096 163.5700694654183 L68.01250518196721 161.55104198127452 L72.02504236084434 164.43536695862278 L76.03757953972146 156.64768951978255 L80.05011671859859 156.07082452431288 L84.06265389747571 155.20552703110843 L88.07519107635284 147.12941709453338 L92.08772825522993 151.1674720628209 L96.10026543410706 147.70628209000301 L100.11280261298418 148.5715795832075 L104.12533979186131 144.8219571126548 L108.13787697073843 137.32271217154937 L112.15041414961556 144.24509211718515 L116.16295132849268 141.07233464210208 L120.1754885073698 137.32271217154937 L124.18802568624693 136.74584717607974 L128.20056286512406 132.13092721232255 L132.21310004400118 130.1118997281788 L136.22563722287828 136.45741467834492 L140.2381744017554 128.9581697372395 L144.25071158063253 128.09287224403505 L148.26324875950965 126.93914225309574 L152.27578593838678 119.72832980972515 L156.2883231172639 123.18951978254304 L160.30086029614102 120.59362730292963 L164.31339747501815 116.26713983690728 L168.32593465389527 111.6522198731501 L172.3384718327724 107.03729990939293 L176.35100901164952 118.28616732105104 L180.36354619052665 113.9596798550287 L184.37608336940374 105.59513742071881 L188.38862054828087 100.403352461492 L192.401157727158 111.94065237088493 L196.41369490603512 98.38432497734824 L200.42623208491224 98.96118997281789 L204.43876926378937 96.07686499546965 L208.451306